# Mesa Data Preparation
So, I've finally decided that there's no way to circumvent the need to use MESA (Modules for Experimental Stellar Astrophysics) to do this research.

This means we will have to prepare the initial simulation data for MESA. We retrieved the stars from the Gaia DR3 dataset using the following ADQL query:

```sql
SELECT TOP 1000000
	gaia.source_id, 
	gaia.ra as ra,
	gaia.dec as dec,
	parameters.mass_flame AS mass,
	parameters.radius_gspphot AS radius,
	parameters.age_flame as age
FROM gaiadr3.astrophysical_parameters AS parameters
INNER JOIN gaiadr3.gaia_source as gaia
ON gaia.source_id = parameters.source_id
WHERE 
	parameters.mass_flame IS NOT NULL
	AND parameters.fem_gspspec IS NOT NULL
	AND parameters.age_flame IS NOT NULL
	AND parameters.evolstage_flame > 100 -- 100 = Zero age main sequence star
	AND parameters.evolstage_flame < 360 -- 360 = main sequence turn off
```

## The Simulations
We will be running an evolutionary simulation for each star up until its current age (estimated by Gaia).
We will provide metallicity, initial mass, age, and initial helium (calculated from the metallicity) as inputs to the simulation.

We will export the simulation parameters to a csv `data.csv` file and use that as input for parallel execution using GNU Parallel.
The data will be exported to the `parallel` directory which will then be used to run the simulations.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from astropy.table import Table
from astropy.io import fits
import os
import requests
import gzip
from collections import namedtuple
from typing import Union
from pathlib import Path

### Import the Gaia Data and convert to a pandas DataFrame
The above query must be run in the [Gaia Archive Advanced Query Tool (ADQL)](https://gea.esac.esa.int/archive/) and then the results must be downloaded and stored in the `data` directory.

First, we must unzip the file, then we can read it into a pandas DataFrame.

In [2]:
filename = 'data/gaia_astrophysical_parameters.vot.gz'

# If unzipped file does not exist, unzip and decode it
if not os.path.exists('data/gaia_astrophysical_parameters.vot'):
    with gzip.open(filename, 'rb') as f:
        with open('data/gaia_astrophysical_params.vot', 'wb') as g:
            g.write(f.read())

In [2]:
df: pd.DataFrame = Table.read('data/gaia_astrophysical_params.vot').to_pandas()
df

,SOURCE_ID,ra,dec,log_surface_gravity,dist,mass,radius,log_fe_h_abundance,log_n_fe_abundance,log_s_fe_abundance,log_metalicity,age,evolution_stage,spectral_type
0,6030024480434041088,256.473786,-28.289514,3.9771,177.857895,1.409851,2.0905,-0.09,0.47,0.11,0.04,2.920355,316,F
1,6030111685419272576,255.038856,-28.811967,4.2980,182.385498,1.171434,1.1288,0.18,NaN,NaN,-0.54,1.242147,160,G
2,5253752950266984704,157.348507,-62.332491,3.7712,279.329987,1.670828,2.9348,-0.19,NaN,0.19,-0.00,1.923422,337,F
3,6030062409175986176,254.932569,-28.931663,3.9315,405.607208,1.477585,2.2852,-0.13,NaN,NaN,0.20,2.654922,327,F
4,5961557062324546304,265.895060,-37.946593,3.9318,342.907501,1.518954,2.1685,0.03,NaN,NaN,0.06,2.108766,277,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205584,1796082381758247552,328.159849,25.054985,4.0129,112.224403,1.343488,1.8720,-0.07,0.02,-0.04,0.25,3.229928,312,F
205585,1794976651018650624,328.512913,22.939668,4.1252,213.870605,1.105674,1.3825,-0.02,NaN,NaN,-0.05,5.438365,337,F
205586,1796123682164850688,326.907823,24.488709,3.9158,355.876312,1.487074,2.3270,-0.16,NaN,0.16,-0.11,2.510785,303,F
205587,1795069284873344640,328.001389,23.488187,4.1705,167.741104,1.119260,1.2895,0.12,NaN,NaN,-0.22,4.767622,320,F


## Spectroscopy Data (APOGEE DR17)
We can improve our MESA simulations by using more fine-grained spectroscopy data.

Download the FITS file from SDSS's (Sloan Digital Sky Survey) [cloud storage](https://data.sdss.org/sas/dr17/apogee/spectro/aspcap/dr17/synspec_rev1/allStar-dr17-synspec_rev1.fits)

This may take a while to download.

In [4]:
url = 'https://data.sdss.org/sas/dr17/apogee/spectro/aspcap/dr17/synspec_rev1/allStar-dr17-synspec_rev1.fits'

if not os.path.exists('data/APOGEE_DR17.fits'):
    response = requests.get(url)
response.raise_for_status()

# Store to 'data/galah_all_stars_spectroscopy.fits'
with open('data/APOGEE_DR17.fits', 'wb') as f:
    f.write(response.content)

In [3]:
# Load the data
cols = ['GAIAEDR3_SOURCE_ID', 'C_FE', 'C_FE_FLAG', 'N_FE', 'N_FE_FLAG', 'O_FE', 'O_FE_FLAG', 'NA_FE', 'NA_FE_FLAG', 'MG_FE', 'MG_FE_FLAG', 'AL_FE', 'AL_FE_FLAG', 'SI_FE', 'SI_FE_FLAG', 'S_FE', 'S_FE_FLAG', 'K_FE', 'K_FE_FLAG', 'CA_FE', 'CA_FE_FLAG', 'CR_FE', 'CR_FE_FLAG', 'MN_FE', 'MN_FE_FLAG', 'FE_H', 'FE_H_FLAG', 'NI_FE', 'NI_FE_FLAG']
tbl: pd.DataFrame = Table.read('data/APOGEE_DR17.fits', format='fits', hdu=1)
apogee_spectroscopy = tbl[cols].to_pandas()

# Neon is not measured since its spectra are in the UV so we will use the average of the [O/Fe] and [Mg/Fe] spectra since they are created in the same fusion processes as Neon
# Suggested by this paper: https://iopscience.iop.org/article/10.3847/1538-3881/ac9bfa#ajac9bfas3
apogee_spectroscopy['NE_FE'] = (apogee_spectroscopy['O_FE'] + apogee_spectroscopy['MG_FE']) / 2
apogee_spectroscopy['NE_FE_FLAG'] = apogee_spectroscopy['O_FE_FLAG'] + apogee_spectroscopy['MG_FE_FLAG'] # Arbitrary flag that will be zero if the both oxygen and magnesium are good data

apogee_spectroscopy

,GAIAEDR3_SOURCE_ID,C_FE,C_FE_FLAG,N_FE,N_FE_FLAG,O_FE,O_FE_FLAG,NA_FE,NA_FE_FLAG,MG_FE,...,CR_FE,CR_FE_FLAG,MN_FE,MN_FE_FLAG,FE_H,FE_H_FLAG,NI_FE,NI_FE_FLAG,NE_FE,NE_FE_FLAG
0,0,0.004847,0,0.124265,0,0.114938,0,0.146668,0,0.035147,...,-0.025774,0,0.040870,0,0.003463,0,0.051278,0,0.075043,0
1,538028216707715712,0.009295,0,0.151220,0,0.083402,0,0.050112,0,0.030429,...,NaN,64,NaN,64,-0.160680,0,0.007683,0,0.056915,0
2,2413929812587459072,0.061738,0,-0.111900,256,0.235343,0,-0.190826,0,0.165238,...,-0.130982,0,-0.078027,0,-0.275530,0,0.013930,0,0.200291,0
3,422596679964513792,0.112730,0,0.114560,32,0.045069,0,-0.179346,0,0.038494,...,-1.112472,0,0.063633,0,-0.252970,0,-0.108750,0,0.041782,0
4,422596679964513792,0.032651,0,-0.489060,288,0.140573,0,-1.166266,0,-0.001095,...,-0.741282,0,-0.153027,0,-0.214170,0,-0.069440,0,0.069739,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733896,2341765776376373376,NaN,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
733897,1998097371124974720,-0.060314,0,0.201880,0,0.088913,0,-0.127408,0,0.083532,...,0.038944,0,-0.000038,0,-0.236560,0,0.022203,0,0.086223,0
733898,1994741318040223232,-0.011308,0,0.209650,0,0.050574,0,0.042082,0,0.064306,...,0.024764,0,0.036437,0,0.114820,0,0.032992,0,0.057440,0
733899,6379914575198998272,-0.616490,0,-0.004530,288,-0.064257,0,0.043602,0,-0.211707,...,0.043914,0,-0.584298,0,-1.050500,0,-0.103507,0,-0.137982,0


In [4]:
## Cross-match the Gaia data with the Galah data (using the `SOURCE_ID` in the Gaia data and the `dr3_source_id` in the Galah data)
apogee_spectroscopy.rename(columns={'GAIAEDR3_SOURCE_ID': 'SOURCE_ID'}, inplace=True)
matched_df = pd.merge(df, apogee_spectroscopy[
    ['SOURCE_ID', 'C_FE', 'C_FE_FLAG', 'O_FE', 'O_FE_FLAG', 'NE_FE_FLAG', 'NE_FE_FLAG', 'MG_FE', 'MG_FE_FLAG', 'AL_FE', 'NE_FE', 'NE_FE_FLAG',
     'AL_FE_FLAG', 'SI_FE', 'SI_FE_FLAG', 'K_FE', 'K_FE_FLAG', 'CA_FE', 'CA_FE_FLAG', 'CR_FE',
     'CR_FE_FLAG', 'MN_FE', 'MN_FE_FLAG', 'NI_FE', 'NI_FE_FLAG', 'FE_H', 'FE_H_FLAG', 'N_FE','N_FE_FLAG', 'S_FE', 'S_FE_FLAG']], on='SOURCE_ID', how='inner')
matched_df.rename(columns={'C_FE': 'C', 'O_FE': 'O', 'NE_FE': 'Ne',
                           'MG_FE': 'Mg', 'AL_FE': 'Al', 'SI_FE': 'Si',
                           'K_FE': 'K', 'CA_FE': 'Ca', 
                           'CR_FE': 'Cr', 'MN_FE': 'Mn', 'NI_FE': 'Ni',
                           'FE_H': 'Fe', 'Ne_Fe': 'Ne',
                           'N_FE_FLAG': 'flag_N',
                           'FE_H_FLAG': 'flag_Fe',
                           'N_FE': 'N', 'S_FE': 'S', 'C_FE_FLAG': 'flag_C',
                           'O_FE_FLAG': 'flag_O', 'NE_FE_FLAG': 'flag_Ne',
                           'MG_FE_FLAG': 'flag_Mg', 'AL_FE_FLAG': 'flag_Al',
                           'SI_FE_FLAG': 'flag_Si', 'K_FE_FLAG': 'flag_K',
                           'CA_FE_FLAG': 'flag_Ca', 'S_FE_FLAG': 'flag_S',
                           'CR_FE_FLAG': 'flag_Cr', 'MN_FE_FLAG': 'flag_Mn',
                           'NI_FE_FLAG': 'flag_Ni'},
                  inplace=True)

matched_df = matched_df.dropna(subset=['Fe', 'N', 'O', 'C', 'Mg', 'Si', 'Ne', 'S'])
matched_df = matched_df[matched_df['flag_Fe'] == 0]
matched_df = matched_df[matched_df['flag_N'] == 0]
matched_df = matched_df[matched_df['flag_O'] == 0]
matched_df = matched_df[matched_df['flag_C'] == 0]
matched_df = matched_df[matched_df['flag_Mg'] == 0]
matched_df = matched_df[matched_df['flag_Si'] == 0]
matched_df = matched_df[matched_df['flag_S'] == 0]

matched_df

,SOURCE_ID,ra,dec,log_surface_gravity,dist,mass,radius,log_fe_h_abundance,log_n_fe_abundance,log_s_fe_abundance,...,Mn,flag_Mn,Ni,flag_Ni,Fe,flag_Fe,N,flag_N,S,flag_S
4,2268469823008193152,282.016475,75.423453,4.5217,145.762207,0.823140,0.8730,-0.00,NaN,NaN,...,0.141202,0,0.094350,0,0.211670,0,0.250330,0,-0.087508,0
8,2264752202396093440,292.152517,72.519192,4.5361,146.394302,0.815044,0.8196,0.01,NaN,NaN,...,0.040761,0,0.014277,0,0.051663,0,0.033958,0,-0.023827,0
18,2268625472622936832,280.909000,76.005640,4.4328,180.245499,0.906775,0.9562,0.00,NaN,NaN,...,0.079832,0,0.038490,0,0.116710,0,0.161760,0,-0.066233,0
19,2266607387750059648,276.122223,72.223313,4.5287,117.208000,0.857668,0.8635,0.05,NaN,NaN,...,0.055690,0,0.053450,0,0.151250,0,0.094051,0,-0.150428,0
42,2265008732201710848,292.041921,74.263128,4.4988,166.162903,0.919691,0.9020,-0.07,NaN,NaN,...,0.042765,0,0.019040,0,0.134380,0,0.033745,0,0.014305,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17014,6051325319082291072,249.930905,-22.299954,4.3875,120.769699,0.894263,1.0553,-0.07,NaN,-0.21,...,0.007943,0,0.041774,0,0.028000,0,0.026942,0,-0.074890,0
17018,6052210387286181632,245.602881,-21.537031,4.5364,123.393303,0.798299,0.8032,-0.00,NaN,NaN,...,0.022232,0,0.022975,0,0.039333,0,0.019557,0,0.054559,0
17038,1175680781122200192,216.943306,8.551573,4.4730,110.827499,0.986199,0.9424,-0.07,0.51,0.10,...,0.019909,0,0.018225,0,0.095665,0,0.117899,0,-0.070877,0
17051,1176284378646107008,217.274190,9.046392,4.3453,56.396900,0.956969,1.0322,-0.11,NaN,-0.15,...,0.044954,0,0.032282,0,0.070278,0,0.180159,0,-0.104771,0


### Necessary Constants

In [28]:
# According to this paper: https://www.aanda.org/articles/aa/full_html/2020/06/aa37694-20/aa37694-20.html
# Gases produced by the big bang were approximately 24.9% helium, still according to the above paper
Y_PROTO = 0.249
Y_SUN = 0.279
Z_SUN = 0.0122

NUM_STARS = 3000 
SUN_HE_ABUNDANCE = 10.93

HYDROGEN_MASS = 1.00784
HELIUM_MASS = 4.002602
CARBON_MASS = 12.011
NITROGEN_MASS = 14.0067
OXYGEN_MASS = 15.999
NEON_MASS = 20.1797
SODIUM_MASS = 22.989769
MAGNESIUM_MASS = 24.305
ALUMINUM_MASS = 26.981539
SILICON_MASS = 28.0855
SUlFUR_MASS = 32.065
POTASSIUM_MASS = 39.0983
CALCIUM_MASS = 40.078
CHROMIUM_MASS = 51.9961
MANGANESE_MASS = 54.938044
IRON_MASS = 55.845
NICKEL_MASS = 58.6934

masses = {
    'H': HYDROGEN_MASS,
    'He': HELIUM_MASS,
    'C': CARBON_MASS,
    'N': NITROGEN_MASS,
    'O': OXYGEN_MASS,
    'Ne': NEON_MASS,
    'Na': SODIUM_MASS,
    'Mg': MAGNESIUM_MASS,
    'Al': ALUMINUM_MASS,
    'Si': SILICON_MASS,
    'S': SUlFUR_MASS,
    'K': POTASSIUM_MASS,
    'Ca': CALCIUM_MASS,
    'Cr': CHROMIUM_MASS,
    'Mn': MANGANESE_MASS,
    'Fe': IRON_MASS,
    'Ni': NICKEL_MASS,
}

atomic_numbers = {
    'H': 1,
    'He': 2,
    'C': 6,
    'N': 7,
    'O': 8,
    'Ne': 10,
    'Na': 11,
    'Mg': 12,
    'Al': 13,
    'Si': 14,
    'S': 16,
    'K': 19,
    'Ca': 20,
    'Cr': 24,
    'Mn': 25,
    'Fe': 26,
    'Ni': 28,
}

SPECTROSCOPY_ATOMS = ['C', 'O', 'N', 'Ne', 'Mg', 'Al', 'Si', 'S', 'K', 'Ca', 'Cr', 'Mn', 'Fe', 'Ni']
ATOMIC_UNIT = 1.6605402E-27  # kg

### Stellar Data Model
The stellar data model is a object that contains the following properties for a star:
- mass (mSun)
- relative abundances
- mass fractions (calculated from relative abundances)
- metallicity (calculated from mass fractions)
- age
- radius (rSun)

A stellar data model is the input for a MESA simulation and contains all necessary information to run a stellar evolution simulation.

In [3]:
Abundance = namedtuple('Abundance', ['atom', 'relAbundance'])  # Atom name, relative abundance
AbundanceVal = namedtuple('AbundanceVal', ['aMass', 'abundance'])

### Solar Baseline Model
The solar data model is used to calculate metallicities and absolute abundances for any given star.

The solar model is based on the abundances of the Sun as calculated in this [paper](https://iopscience.iop.org/article/10.3847/1538-3881/ac9bfa#ajac9bfas3).

In [34]:
class SunModel:
    def __init__(self):
        self.mass = 1
        self.abundances = {
            'H': AbundanceVal(HYDROGEN_MASS, 12),
            'He': AbundanceVal(HELIUM_MASS, SUN_HE_ABUNDANCE),
            'C': AbundanceVal(CARBON_MASS, 8.39),
            'N': AbundanceVal(NITROGEN_MASS, 7.78),
            'O': AbundanceVal(OXYGEN_MASS, 8.66),
            'Ne': AbundanceVal(NEON_MASS, 7.84),
            'Na': AbundanceVal(SODIUM_MASS, 6.17),
            'Mg': AbundanceVal(MAGNESIUM_MASS, 7.53),
            'Al': AbundanceVal(ALUMINUM_MASS, 6.37),
            'Si': AbundanceVal(SILICON_MASS, 7.51),
            'S': AbundanceVal(SUlFUR_MASS, 7.14),
            'K': AbundanceVal(POTASSIUM_MASS, 5.08),
            'Ca': AbundanceVal(CALCIUM_MASS, 6.31),
            'Cr': AbundanceVal(CHROMIUM_MASS, 5.64),
            'Mn': AbundanceVal(MANGANESE_MASS, 5.39),
            'Fe': AbundanceVal(IRON_MASS, 7.45),
            'Ni': AbundanceVal(NICKEL_MASS, 6.23),
        }
        self.age = 4.57e9
        self.radius = 1.0
        self.mass_fracs = self.get_mass_fracs()

    def get_mass_fracs(self):
        return {
            'y': Y_SUN,
            'z': Z_SUN
        }

    @property
    def z_frac(self):
        return self.mass_fracs['z']

    @property
    def y_frac(self):
        return self.mass_fracs['y']

    @property
    def abundance_np(self):
        _pyarr = []
        for (atom, abundance) in self.abundances.items():
            _pyarr.append((abundance.abundance, abundance.aMass, atomic_numbers[atom]))

        return np.array(_pyarr)

### Stellar Data Model
The stellar data model represents a star's simulation data.

In [51]:
# Solar model and calculation constants
sun_model = SunModel()
sun_abundances = sun_model.abundance_np
sun_metals = sun_abundances[2:]
sun_metal = np.sum(10 ** sun_metals[:, 0] * sun_metals[:, 2])

948595122.1161544


array([[ 8.39    , 12.011   ,  6.      ],
       [ 7.78    , 14.0067  ,  7.      ],
       [ 8.66    , 15.999   ,  8.      ],
       [ 7.84    , 20.1797  , 10.      ],
       [ 6.17    , 22.989769, 11.      ],
       [ 7.53    , 24.305   , 12.      ],
       [ 6.37    , 26.981539, 13.      ],
       [ 7.51    , 28.0855  , 14.      ],
       [ 7.14    , 32.065   , 16.      ],
       [ 5.08    , 39.0983  , 19.      ],
       [ 6.31    , 40.078   , 20.      ],
       [ 5.64    , 51.9961  , 24.      ],
       [ 5.39    , 54.938044, 25.      ],
       [ 7.45    , 55.845   , 26.      ],
       [ 6.23    , 58.6934  , 28.      ]])

In [60]:
SOLAR_MASS = 1.989 * 10 ** 30  # kg

class StellarDataModel:
    def __init__(self, mass: float, relative_abundances: list[Abundance], age: float, radius: float, rel_iron: float,
                 iters: Union[int, 'converge'] = 'converge', convergence_thresh: float = 1e-10):
        """
        :param mass: Mass of star in MSun 
        :param relative_abundances: List of relative abundances for each element
        :param age: Age of star in Gya
        :param rel_iron: Relative abundance of iron
        :param radius: Radius of star in RSun
        :param iters: Iteration number or 'converge' to use the convergence criteria
        :param convergence_thresh: Convergence threshold (difference between iterations must be less than this)
        """
        self.mass = mass
        self.abundances = StellarDataModel.get_abundances(relative_abundances, rel_iron)
        self.age = age
        self.mass_fracs = self.get_mass_fracs(iters, convergence_thresh)
        self.radius = radius

    @property
    def y_frac(self):
        return self.mass_fracs['y']

    @property
    def z_frac(self):
        return self.mass_fracs['z']

    # Iteratively calculate the mass fractions for each element
    def get_mass_fracs(self, iters: Union[int, 'converge'] = 'converge', convergence_thresh: float = 1e-10):
        idx_to_atom = {}
        _pyarr = []
        for (atom, abundance) in self.abundances.items():
            _pyarr.append((abundance.abundance, abundance.aMass, atomic_numbers[atom]))
            idx_to_atom[len(_pyarr) - 1] = atom

        arr = np.array(_pyarr)

        mass = self.mass * SOLAR_MASS
        metal_star = np.sum(10 ** arr[:, 0] * arr[:, 2])
        delta_z_frac = metal_star / sun_metal - 1
        delta_helium = 2.1 * delta_z_frac * Z_SUN
        helium_mass_fraction = delta_helium + Y_SUN
        z =  delta_z_frac * Z_SUN + Z_SUN
        
        num_atoms_helium = helium_mass_fraction * mass / (HELIUM_MASS * ATOMIC_UNIT)
        num_atoms_hydrogen = (1 - helium_mass_fraction - z) * mass / (HYDROGEN_MASS * ATOMIC_UNIT)
        
        # same as np.log10((num_atoms_helium / num_atoms_hydrogen) * 10**12), which is helium to hydrogen ratio converted to helium per 10^12 hydrogen
        abundance_helium = 12 + np.log10(num_atoms_helium / num_atoms_hydrogen) 
        stack_args = [
            np.array([
                (12, HYDROGEN_MASS, 1),
                (abundance_helium, HELIUM_MASS, 2),
            ]),
            arr,
        ]
        
        star_new = np.vstack(stack_args)

        f_H_star = np.log10(np.sum(10 ** star_new[:, 0] * star_new[:, 2]))

        # Ratio of number of nucleons of element to all nucleons, Y_Q
        # While not explicity stated in the paper, before calculating the mass fractions, we need to convert from log10 to unlogged form
        # This was verified by looking at Table 1 in the paper and manually calculating the mass fraction by keeping the nucleon fraction logged and unlogging the nucleon fractions
        atom_fracs = 10 ** (arr[:, 0] - f_H_star)
        
        # Mass fractions, X_Q
        mass_fracs = atom_fracs * arr[:, 1]
        
        # Z value
        metallicity = np.sum(mass_fracs)
        
        def calc_z_iter(arr: np.ndarray, z: float) -> float:
            delta_z = z - sun_model.z_frac
            delta_helium = 2.1 * delta_z
            helium_mass_fraction = delta_helium + Y_SUN
            
            num_atoms_helium = helium_mass_fraction * mass / (HELIUM_MASS * ATOMIC_UNIT)
            num_atoms_hydrogen = (1 - helium_mass_fraction - z) * mass / (HYDROGEN_MASS * ATOMIC_UNIT)

            abundance_helium = 12 + np.log10(num_atoms_helium / num_atoms_hydrogen)
            stack_args = [
                np.array([
                    (12, HYDROGEN_MASS, 1),
                    (abundance_helium, HELIUM_MASS, 2),
                ]),
                arr,
            ]

            star_new = np.vstack(stack_args)

            f_H_star = np.log10(np.sum(10 ** star_new[:, 0] * star_new[:, 2]))

            atom_fracs = 10 ** (arr[:, 0] - f_H_star)
            mass_fracs = atom_fracs * arr[:, 1]
            metallicity = np.sum(mass_fracs)
            return metallicity, mass_fracs

        z_last = float('inf')
        z_curr = metallicity
        
        if iters == 'converge':
            iters = 0

            while np.abs(z_curr - z_last) > convergence_thresh:
                z_last = z_curr
                z_curr, mass_fracs = calc_z_iter(arr, z_last)
                iters += 1

            delta_z = z_curr - sun_model.z_frac
            delta_helium = 2.1 * delta_z
            
            y_frac = delta_helium + Y_SUN

            mass_fractions = {}
            for i in range(len(mass_fracs)):
                mass_fractions[idx_to_atom[i]] = mass_fracs[i]

            mass_fractions['y'] = y_frac
            mass_fractions['z'] = z_curr

            return mass_fractions
        else:
            for i in range(iters - 1):
                z_last = z_curr
                z_curr, mass_fracs = calc_z_iter(arr, z_last)
            
            delta_z = z_curr - sun_model.z_frac
            delta_helium = 2.1 * delta_z
            y_frac = delta_helium + Y_SUN

            mass_fractions = {}
            for i in range(len(mass_fracs)):
                mass_fractions[idx_to_atom[i]] = mass_fracs[i]

            mass_fractions['y'] = y_frac
            mass_fractions['z'] = z_curr

            return mass_fractions

    @staticmethod
    def get_abundances(relative_abundances: list[Abundance], iron_abundance) -> dict[str, AbundanceVal]:
        abundances = {}
        for rel_ab in relative_abundances:
            # Q/Fe = Q/H - Fe/H; Thus Q/H = Fe/H + Q/Fe
            if rel_ab.atom == 'Fe':
                converted_as_base_hydrogen = iron_abundance
            else:
                converted_as_base_hydrogen = iron_abundance + rel_ab.relAbundance
            
            abundances[rel_ab.atom] = AbundanceVal(masses[rel_ab.atom],
                                                   converted_as_base_hydrogen + sun_model.abundances[rel_ab.atom].abundance)

        return abundances

    def to_mesa_data(self):
        data = {
            'mass': self.mass,
            'y': self.y_frac,
            'z': self.z_frac,
            'age': self.age * 1e9,
            'radius': self.radius,
        }

        for (atom, mass_frac) in self.mass_fracs.items():
            data[f'{atom}_mass_frac'] = mass_frac

        return data

In [58]:
model = StellarDataModel(1, [
    Abundance('C', .03),
    Abundance('N', -.19),
    Abundance('O', -.04),
    Abundance('Ne', .16),
    Abundance('Na', 0),
    Abundance('Mg', -.08),
    Abundance('Al', -0.03),
    Abundance('Si', -0.01),
    Abundance('S', -0.07),
    Abundance('K', 0),
    Abundance('Ca', -0.01),
    Abundance('Cr', -.19),
    Abundance('Mn', 0),
    Abundance('Fe', .2),
    Abundance('Ni', -0.07),
], 4.603, 1, .2)
print(sun_metal)
model.abundances
# model.to_mesa_data()

1468078981.5114636
0.5476349680529973
12.04481342090511
948595122.1161544
{'C': 0.00406366096140127, 'N': 0.0007009291582059978, 'O': 0.008578890770004443, 'Ne': 0.002595691084213399, 'Na': 4.3739408671843046e-05, 'Mg': 0.0008811176995157752, 'Al': 7.59285344973671e-05, 'Si': 0.0011424059589312343, 'S': 0.00048458452843746683, 'K': 6.04639025276308e-06, 'Ca': 0.00010285948386953024, 'Cr': 1.884990084994532e-05, 'Mn': 1.7346485311053117e-05, 'Fe': 0.002024522441039229, 'Ni': 0.00010912583201847912, 'y': 0.2971559671381616, 'z': 0.0208456986372198}


{'C': AbundanceVal(aMass=12.011, abundance=8.620000000000001),
 'N': AbundanceVal(aMass=14.0067, abundance=7.79),
 'O': AbundanceVal(aMass=15.999, abundance=8.82),
 'Ne': AbundanceVal(aMass=20.1797, abundance=8.2),
 'Na': AbundanceVal(aMass=22.989769, abundance=6.37),
 'Mg': AbundanceVal(aMass=24.305, abundance=7.65),
 'Al': AbundanceVal(aMass=26.981539, abundance=6.54),
 'Si': AbundanceVal(aMass=28.0855, abundance=7.7),
 'S': AbundanceVal(aMass=32.065, abundance=7.27),
 'K': AbundanceVal(aMass=39.0983, abundance=5.28),
 'Ca': AbundanceVal(aMass=40.078, abundance=6.5),
 'Cr': AbundanceVal(aMass=51.9961, abundance=5.6499999999999995),
 'Mn': AbundanceVal(aMass=54.938044, abundance=5.59),
 'Fe': AbundanceVal(aMass=55.845, abundance=7.65),
 'Ni': AbundanceVal(aMass=58.6934, abundance=6.36)}

In [34]:
# Load past data to exclude from the available stars (don't want to simulate stars that have already been simulated)
glob = Path('simulate/past_data/').glob('*.csv')
dfs = []
for file in glob:
    dfs.append(pd.read_csv(file, index_col=0))

if len(dfs) > 0:
    past_data = pd.concat(dfs)
else:
    past_data = pd.DataFrame()

# Remove the stars that have already been simulated
available_stars = matched_df[~matched_df.SOURCE_ID.isin(past_data.index)]
available_stars = available_stars.drop_duplicates(subset=['SOURCE_ID'])
available_stars

NameError: name 'matched_df' is not defined

### Prepare Dataset
Now we will prepare the dataset for MESA.

First, we will need to convert the age to a time in years. (Convert from Gya to years)
Next, we will need to calculate the initial helium abundance from the metallicity. (y)
Finally, we will add the mass and metallicity to the dataframe. (mass and z respectively)

We will also include the source_id as a column in the dataframe.

In [13]:
def format_test(test_bool):
    return '✔' if test_bool else '✘'


cols = ['source_id', 'mass', 'age', 'y', 'z', 'radius']
for atom in SPECTROSCOPY_ATOMS:
    cols.append(f'{atom}_mass_frac')
dataset = pd.DataFrame(columns=cols)
dataset.set_index('source_id', inplace=True)
df_len = len(available_stars)

# Inference test conditions
print(
    f'The sample is independent if the sample size is less than {df_len / 10}. {NUM_STARS} is less than {df_len / 10}? {format_test(NUM_STARS < df_len)}')
print(
    f'The sample is normal if the sample size is greater than 30 or the sampling or population distribution is normal. We do not know the density profiles of the stars so we will have to rely on the Central Limit Theorem. {NUM_STARS} is greater than 30? {format_test(NUM_STARS > 30)}')
print(f'The sample is random because we used NumPy random to select indices. ✔')

print()
print(f'The steps for inference have been met? {format_test(NUM_STARS < df_len / 10 and NUM_STARS > 30)}')

if NUM_STARS < df_len:
    random_indices = np.random.choice(df_len, NUM_STARS, replace=False)
else:
    random_indices = np.arange(df_len)

for i in random_indices:
    df_row = available_stars.iloc[i]

    abundances: list[Abundance] = []
    # Create stellar data model
    
    for atom in SPECTROSCOPY_ATOMS:
        val = df_row[f'{atom}']
        flag = df_row[f'flag_{atom}']
        if atom == 'Ne':
            flag = df_row['flag_Ne'].iloc[0]
        if (not np.isnan(val) 
                and flag == 0):
            abundances.append(Abundance(atom, val))
    
    # Create stellar data model
    stellar_data_model = StellarDataModel(df_row['mass'], abundances, df_row['age'], df_row['radius'], df_row['Fe'])
    dataset.loc[df_row.SOURCE_ID] = stellar_data_model.to_mesa_data()
    
    if dataset.loc[df_row.SOURCE_ID] is None:
        print(f'Failed to create stellar data model for {df_row.SOURCE_ID}')
    
dataset

The sample is independent if the sample size is less than 189.5. 3000 is less than 189.5? ✘
The sample is normal if the sample size is greater than 30 or the sampling or population distribution is normal. We do not know the density profiles of the stars so we will have to rely on the Central Limit Theorem. 3000 is greater than 30? ✔
The sample is random because we used NumPy random to select indices. ✔

The steps for inference have been met? ✘


,mass,age,y,z,radius,C_mass_frac,O_mass_frac,N_mass_frac,Ne_mass_frac,Mg_mass_frac,Al_mass_frac,Si_mass_frac,S_mass_frac,K_mass_frac,Ca_mass_frac,Cr_mass_frac,Mn_mass_frac,Fe_mass_frac,Ni_mass_frac
source_id,,,,,,,,,,,,,,,,,,,
2268469823008193152,0.823140,1.299812e+10,0.273147,0.018156,0.8730,0.003306,0.007353,0.001585,0.001490,0.000933,0.000088,0.001114,0.000382,0.000005,0.000077,0.000013,0.000020,0.001661,0.000131
2264752202396093440,0.815044,1.114927e+10,0.265868,0.012683,0.8196,0.001799,0.005622,0.000745,0.001140,0.000714,0.000068,0.000784,0.000342,0.000004,0.000068,0.000015,0.000012,0.001285,0.000084
2268625472622936832,0.906775,9.327484e+09,0.269961,0.015761,0.9562,0.002518,0.007094,0.001089,0.001340,0.000782,0.000072,0.000931,0.000338,0.000005,0.000065,0.000016,0.000014,0.001400,0.000097
2266607387750059648,0.857668,9.406262e+09,0.270369,0.016067,0.8635,0.002606,0.007081,0.001003,0.001413,0.000872,0.000074,0.000990,0.000300,0.000004,0.000073,0.000022,0.000015,0.001506,0.000108
2265008732201710848,0.919691,6.044249e+09,0.269255,0.015230,0.9020,0.002136,0.007036,0.000854,0.001328,0.000774,0.000066,0.000920,0.000428,0.000004,0.000073,0.000024,0.000014,0.001474,0.000098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6051325319082291072,0.894263,1.307046e+10,0.267790,0.014128,1.0553,0.002010,0.007061,0.000673,0.001210,0.000640,0.000063,0.000846,0.000279,0.000002,0.000056,0.000013,0.000010,0.001180,0.000082
6052210387286181632,0.798299,1.202229e+10,0.266512,0.013167,0.8032,0.002237,0.005692,0.000693,0.001147,0.000714,0.000059,0.000825,0.000394,0.000003,0.000060,0.000012,0.000011,0.001236,0.000083
1175680781122200192,0.986199,3.324886e+09,0.267498,0.013908,0.9424,0.001827,0.006414,0.000975,0.001191,0.000683,0.000062,0.000843,0.000331,0.000004,0.000070,0.000018,0.000012,0.001386,0.000092


In [14]:
# In case you want to scout out the original rows
available_stars.iloc[random_indices]

,SOURCE_ID,ra,dec,log_surface_gravity,dist,mass,radius,log_fe_h_abundance,log_n_fe_abundance,log_s_fe_abundance,...,Mn,flag_Mn,Ni,flag_Ni,Fe,flag_Fe,N,flag_N,S,flag_S
4,2268469823008193152,282.016475,75.423453,4.5217,145.762207,0.823140,0.8730,-0.00,NaN,NaN,...,0.141202,0,0.094350,0,0.211670,0,0.250330,0,-0.087508,0
8,2264752202396093440,292.152517,72.519192,4.5361,146.394302,0.815044,0.8196,0.01,NaN,NaN,...,0.040761,0,0.014277,0,0.051663,0,0.033958,0,-0.023827,0
18,2268625472622936832,280.909000,76.005640,4.4328,180.245499,0.906775,0.9562,0.00,NaN,NaN,...,0.079832,0,0.038490,0,0.116710,0,0.161760,0,-0.066233,0
19,2266607387750059648,276.122223,72.223313,4.5287,117.208000,0.857668,0.8635,0.05,NaN,NaN,...,0.055690,0,0.053450,0,0.151250,0,0.094051,0,-0.150428,0
42,2265008732201710848,292.041921,74.263128,4.4988,166.162903,0.919691,0.9020,-0.07,NaN,NaN,...,0.042765,0,0.019040,0,0.134380,0,0.033745,0,0.014305,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17014,6051325319082291072,249.930905,-22.299954,4.3875,120.769699,0.894263,1.0553,-0.07,NaN,-0.21,...,0.007943,0,0.041774,0,0.028000,0,0.026942,0,-0.074890,0
17018,6052210387286181632,245.602881,-21.537031,4.5364,123.393303,0.798299,0.8032,-0.00,NaN,NaN,...,0.022232,0,0.022975,0,0.039333,0,0.019557,0,0.054559,0
17038,1175680781122200192,216.943306,8.551573,4.4730,110.827499,0.986199,0.9424,-0.07,0.51,0.10,...,0.019909,0,0.018225,0,0.095665,0,0.117899,0,-0.070877,0
17051,1176284378646107008,217.274190,9.046392,4.3453,56.396900,0.956969,1.0322,-0.11,NaN,-0.15,...,0.044954,0,0.032282,0,0.070278,0,0.180159,0,-0.104771,0


In [15]:
# Export the data to a csv file
dataset.to_csv('simulate/data.csv')

### Setting up for simulations
Please be aware that you will need to download and install both GNU Parallel and MESA.

You can install GNU Parallel using `sudo dnf install parallel` or `sudo apt install parallel`, depending on your distribution.

Please follow the instructions on [MESA's website](https://docs.mesastar.org/en/latest/installation.html).

I used SDK Version **23.7.3** and MESA Version **24.03.1**.

I would highly recommend downloading the source directly from the [MESA GitHub](https://github.com/MESAHub/mesa/releases/tag/r24.03.1) 
rather than downloading it from Zenodo since it can be **slow** to download from Zenodo.

I used a **Fedora 40** distribution tailored for astronomy. I downloaded the ISO image from [here](https://fedoraproject.org/labs/astronomy/).